# SVM Model Training

In [ ]:
# The purpose of this module is to recompute to SVM model (for the out-of-distribution 
# detection function based on the conv 2 layer activations) directly on the inference platform.
# The SVM model generated using ColabPro was generated using a recent version of sklearn that
# was not compatible with the version available on the inference platform.
# I chose to retrain the SVM on the Jetbor rather than downgrade Colab Pro to an older version of sklearn.

### Import required libraries

In [ ]:
import torch
import pickle
import sklearn
from sklearn.svm import OneClassSVM
import os
import collections

### Retrieve activation data

In [ ]:
os.chdir('/home/jetbot/Notebooks/Kashiko')

In [ ]:
# Retrieve activation data recorded during the SVM modle training in ColabPro (see discriminator module)
ACT1 = collections.defaultdict(list)
ACT1 = torch.load('SVM_training_activations1_no_zip.pt')
for k,v in ACT1.items():
    print (k, v.size())

### Train SVM model

Below code reused from https://github.com/gietema/ood-early-layer-detection

"Detecting Out-of-Distribution Inputs in Deep Neural Networks Using an Early-Layer Output" Vahdat Abdelzad, Krzysztof Czarnecki, Rick Salay, Taylor Denounden, Sachin Vernekar, Buu Phan https://arxiv.org/abs/1910.10307

In [ ]:
# Define function to compute the mean of a given channel
def get_mean_channels(batched_outputs):
    channel_means = []
    for single_output in batched_outputs:
        channel_means.append([channel.mean() for channel in single_output])
    return torch.tensor(channel_means)

In [ ]:
batched_output = get_mean_channels(ACT1['conv2'])

In [ ]:
# Retrain SVM with Conv2 activations directly on the Jetbot
SVM_model = OneClassSVM(nu = 0.001).fit(batched_output)

### Save SVM model

In [ ]:
pickle.dump(SVM_model, open('SVM_model.sav', 'wb'))